<a href="https://colab.research.google.com/github/ismaelllamur013/HR2-BotTelegramAudio/blob/main/Reconocimiento_de_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Descripcion rapida:**

Esto es un pequeño notebook en donde buscamos entrenar de forma muy precaria y rapidamente un modelo de deteccion de audio usando una maquina de soportes vectoriales. Basicamente lo que buscamos obtener al final es un modelo para ser usado en la aplicacion que queramos.

A priori lo que se hizo es que a partir de un video, se extraiga su audio.

Para su uso en el modelo remuestreamos a una frecuencia de 44100 Hz, el doble del oido humano, esto para definir una misma frecuencia para todos los audios utilizados para entrenar al modelo.

Posteriormente se busco que todos los audios trabajen con un mismo tamaño, esto para que al modelo esten entrando datos de igual dimenciones

Para modelos de audio agregamos ruido blanco en algunos casos para evaluar que tan bien puede seguir prediciendo nuestro modelo segun cambiamos la relacion señal ruido

Con todo el preprocesamiento definido entrenamos el modelo, evaluaciamos y si es de utilidad, reentrenamos con todos los datos y obtenemos nuestro modelo para usar

# Extaer audio del video


In [ ]:
pip install moviepy

In [ ]:
from moviepy.editor import VideoFileClip

def extraer_audio(input_video, output_audio):
    # Cargar el archivo de video
    video_clip = VideoFileClip(input_video)

    # Extraer el audio del video
    audio_clip = video_clip.audio

    # Guardar el audio en un archivo separado
    audio_clip.write_audiofile(output_audio)

    # Cerrar los clips para liberar recursos
    audio_clip.close()
    video_clip.close()

In [ ]:
# Ejemplo de uso
input_video = "NoSoda12.mp4"
output_audio = "audio_extraido16.wav"
extraer_audio(input_video, output_audio)

MoviePy - Writing audio in audio_extraido16.wav


MoviePy - Done.


# Remuestreo de las señales de audio

In [ ]:
pip install pydub

In [ ]:
from pydub import AudioSegment

def remuestrear_audio(input_audio, output_audio, nueva_frecuencia):
    # Cargar el archivo de audio
    audio = AudioSegment.from_file(input_audio)

    # Realizar el re-muestreo a la nueva frecuencia
    audio_remuestreado = audio.set_frame_rate(nueva_frecuencia)

    # Guardar el audio re-muestreado en un nuevo archivo
    audio_remuestreado.export(output_audio, format="wav")


In [ ]:
# Ejemplo de uso
input_audio = "audio_extraido16.wav"
output_audio_remuestreado = "audio_remuestreado16.wav"
nueva_frecuencia = 44100  # Puedes ajustar la frecuencia según tus necesidades

remuestrear_audio(input_audio, output_audio_remuestreado, nueva_frecuencia)

# Datos con el mismo tamaño

In [ ]:
pip install librosa

In [ ]:
import librosa
import numpy as np
from scipy.io import wavfile

def ajustar_longitud_audio(audio_path, longitud_deseada):
    # Cargar el archivo de audio
    audio, sample_rate = librosa.load(audio_path, sr=None)

    # Ajustar la longitud del audio
    if len(audio) > longitud_deseada:
        audio_ajustado = audio[:longitud_deseada]
    elif len(audio) < longitud_deseada:
        padding = longitud_deseada - len(audio)
        audio_ajustado = np.pad(audio, (0, padding), mode='constant')
    else:
        audio_ajustado = audio

    return audio_ajustado, sample_rate

In [ ]:
# Longitud deseada para todos los archivos de audio (ajusta según tus necesidades)
longitud_deseada = 44100*30 # Ejemplo: 1 segundo a 44.1 kHz

# Ruta del archivo de entrada y salida (ajusta según tus necesidades)
audio_input_path = "audio_remuestreado16.wav"
audio_output_path = "audio_fin16.wav"

# Cargar y ajustar la longitud del audio
audio_ajustado, sample_rate = ajustar_longitud_audio(audio_input_path, longitud_deseada)

# Exportar el audio ajustado a un nuevo archivo WAV
wavfile.write(audio_output_path, sample_rate, audio_ajustado)

# Generador de ruido en los audios


In [ ]:
import librosa
import numpy as np
import soundfile as sf

# Cargar el archivo de audio original
audio_path = "audio_fin4.wav"
y, sr = librosa.load(audio_path, sr=None)

# Generar ruido blanco del mismo tamaño que el audio original
noise = np.random.normal(0, 0.005, len(y))  # Puedes ajustar la amplitud del ruido según tus necesidades

# Mezclar el audio original con el ruido
mixed_audio = y + noise*20

# Guardar el audio mezclado en un nuevo archivo WAV
output_path = "audio_fin4_2.wav"
sf.write(output_path, mixed_audio, sr)


# Predecir

In [ ]:
pip install scikit-learn

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from scipy.io import wavfile  # Para leer archivos de audio WAV

In [ ]:
def extraer_caracteristicas(audio_path):
    # Implementa aquí la extracción de características del audio
    # Puedes utilizar bibliotecas como librosa para extraer características
    # Aquí, se utiliza un ejemplo simple, considerando la longitud del audio
    _, audio_data = wavfile.read(audio_path)
    #longitud_audio = len(audio_data)
    #return np.array([longitud_audio])
    return audio_data

In [ ]:
# Generar datos de ejemplo (reemplaza esto con tus propios datos)
audio_paths_tipo1 = ["audio_fin1.wav", "audio_fin2.wav", "audio_fin3.wav", "audio_fin4.wav","audio_fin1_1.wav", "audio_fin2_1.wav", "audio_fin3_1.wav", "audio_fin4_1.wav","audio_fin1_2.wav", "audio_fin2_2.wav", "audio_fin3_2.wav", "audio_fin4_2.wav"]
audio_paths_tipo2 = ["audio_fin5.wav", "audio_fin6.wav", "audio_fin7.wav", "audio_fin8.wav","audio_fin9.wav", "audio_fin10.wav", "audio_fin11.wav", "audio_fin12.wav","audio_fin13.wav", "audio_fin14.wav", "audio_fin15.wav", "audio_fin16.wav"]

X_tipo1 = np.array([extraer_caracteristicas(path) for path in audio_paths_tipo1])
X_tipo2 = np.array([extraer_caracteristicas(path) for path in audio_paths_tipo2])

In [ ]:
# Etiquetas de clases
y_tipo1 = np.ones(len(X_tipo1))  # Tipo 1
y_tipo2 = np.zeros(len(X_tipo2))  # Tipo 2

In [ ]:
# Concatenar datos y etiquetas
X = np.concatenate([X_tipo1, X_tipo2])
y = np.concatenate([y_tipo1, y_tipo2])

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
# Crear y entrenar el clasificador SVM
classifier = make_pipeline(StandardScaler(), SVC(kernel='linear', C=1))
classifier.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=1, kernel='linear'))])

In [ ]:
# Realizar predicciones en el conjunto de prueba
y_pred = classifier.predict(X_test)

# Evaluar la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy * 100:.2f}%')

Precisión del modelo: 100.00%


In [ ]:
# Crear y entrenar un modelo SVM de clasificación
modelo_svm = SVC(kernel='linear')
modelo_svm.fit(X, y)

# Obtener los parámetros del modelo
coeficientes = modelo_svm.coef_
intercepto = modelo_svm.intercept_

# Imprimir los parámetros
print("Coeficientes:", coeficientes)
print("Intercepto:", intercepto)

Coeficientes: [[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.04205524e-05
  2.07134433e-05 1.97532943e-05]]
Intercepto: [-0.43110799]


In [ ]:
import numpy as np

# ... (Código anterior para obtener coeficientes e intercepto)

# Nuevos datos de entrada para hacer predicciones
nuevos_datos = X_test[0]

# Calcular el valor de decisión sin utilizar el objeto modelo
decision_values = np.dot(coeficientes, nuevos_datos.T) - intercepto

# Aplicar la función de signo para obtener las predicciones (1 para clase positiva, -1 para clase negativa)
predicciones = np.sign(decision_values)

# Imprimir las predicciones
print("Predicciones:", predicciones)

Predicciones: [-1.]


In [ ]:
# Ruta del archivo de texto
ruta_archivo = "coeficientes.txt"
np.savetxt(ruta_archivo, coeficientes)
# Ruta del archivo de texto
ruta_archivo = "intercepto.txt"
np.savetxt(ruta_archivo, intercepto)


In [ ]:
print("Coeficientes:", coeficientes)
print("Intercepto:", intercepto)

Coeficientes: [[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.04205524e-05
  2.07134433e-05 1.97532943e-05]]
Intercepto: [-0.43110799]
